In [204]:
# === Importación de librerías ===
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import classification_report, confusion_matrix, f1_score
import pandas as pd

In [205]:
import pandas as pd

# Carga individual
df_primera = pd.read_csv('D:/TFG/codigo/datos/indicadores-filtrados-primera-5-cambios.csv', sep=",")
df_premier = pd.read_csv('D:/TFG/codigo/datos/indicadores-filtrados-premier-5-cambios.csv', sep=",")
df_bundesliga = pd.read_csv('D:/TFG/codigo/datos/indicadores-filtrados-bundesliga-5-cambios.csv', sep=",")


# Unir en un solo DataFrame
df = pd.concat([df_primera, df_premier, df_bundesliga], ignore_index=True)


In [206]:
# === Limpieza de columnas innecesarias ===
df.drop(['jornada', 'temporada', 'id_indicadores_equipo_prepartido', 'id_partido'], axis=1, inplace=True)

In [207]:
# === Separación de variables predictoras (X) y objetivo (y) ===
X = df.drop(['resultado_partido', 'resultado_local', 'resultado_visitante'], axis=1)

y_resultado = df['resultado_partido']
y_resultado_local = df['resultado_local']
y_resultado_visitante = df['resultado_visitante']

In [208]:
# === Escalado de las variables predictoras entre 0 y 1 ===
scaler = MinMaxScaler()
X_scaled = scaler.fit_transform(X)

### PREDICCIONES RESULTADO 

In [209]:
# === División del dataset en entrenamiento y prueba (80% / 20%) ===
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y_resultado, test_size=0.2, random_state=42)

In [210]:
# === Definición del modelo de árbol de decisión ===
model = DecisionTreeClassifier(random_state=42)

In [211]:
# === Definición del grid de hiperparámetros para optimización ===
param_grid = {
    'max_depth': [3, 5, 7],
    'min_samples_split': [2, 5, 10]
}

In [212]:
# === Búsqueda de los mejores hiperparámetros con validación cruzada ===
grid_search = GridSearchCV(model, param_grid, cv=5, scoring='accuracy', n_jobs=-1)
grid_search.fit(X_train, y_train)

GridSearchCV(cv=5, estimator=DecisionTreeClassifier(random_state=42), n_jobs=-1,
             param_grid={'max_depth': [3, 5, 7],
                         'min_samples_split': [2, 5, 10]},
             scoring='accuracy')

In [213]:
# === Extracción y visualización de los mejores hiperparámetros ===
best_params = grid_search.best_params_
print("Mejores hiperparámetros:", best_params)

Mejores hiperparámetros: {'max_depth': 3, 'min_samples_split': 2}


In [214]:
# === Entrenamiento del modelo con los mejores hiperparámetros ===
best_model = DecisionTreeClassifier(**best_params, random_state=42)
best_model.fit(X_train, y_train)

DecisionTreeClassifier(max_depth=3, random_state=42)

In [215]:
# === Evaluación del modelo sobre el conjunto de prueba ===
y_test_pred = best_model.predict(X_test)
test_accuracy = accuracy_score(y_test, y_test_pred)

In [216]:
# === Cálculo de la importancia de las variables ===
feature_names = X.columns
importances = best_model.feature_importances_

feature_importance_df = pd.DataFrame({
    'Variable': feature_names,
    'Importancia': importances
}).sort_values(by='Importancia', ascending=False)

In [217]:
# === Mostrar las 10 variables más importantes ===
print(feature_importance_df.head(10))

                                              Variable  Importancia
1                  porcentaje local ganados en general     0.278709
82              proporcion visitante posesion en sitio     0.208978
21                 porcentaje local mas 2,5 en general     0.131884
84               proporcion local total tiros en sitio     0.119427
103  proporcion visitante cambios amarillas en general     0.091762
65   porcentaje visitante mas 1,5 encajados en general     0.088547
136  proporcion local cambios defensas a centrocamp...     0.080694
90       proporcion visitante corners a favor en sitio     0.000000
115         media visitante cambios minutos en general     0.000000
109     proporcion local cambios asistentes en general     0.000000


In [218]:
# === Suma de la importancia de variables relacionadas con la rotación ===
cambio_vars = feature_importance_df[feature_importance_df['Variable'].str.contains('cambio', case=False)]
suma_importancia_cambio = cambio_vars['Importancia'].sum()
print(f"Suma total de importancia de las variables relacionadas con la rotación: {suma_importancia_cambio:.4f}")

Suma total de importancia de las variables relacionadas con la rotación: 0.1725


In [219]:
# Matriz de confusión
# Calculamos la matriz de confusión para evaluar los aciertos por clase
conf_matrix = confusion_matrix(y_test, y_test_pred)
print("Matriz de confusión:")
print(conf_matrix)

# F1-score promedio (macro: media sin ponderar entre clases)
f1 = f1_score(y_test, y_test_pred, average='macro')
print(f"F1-score (macro): {f1:.4f}")

# Informe completo con precisión, recall y F1 por clase
print("Informe de clasificación completo:")
print(classification_report(y_test, y_test_pred))

Matriz de confusión:
[[105  45  20]
 [ 50  34  14]
 [ 56  23   7]]
F1-score (macro): 0.3338
Informe de clasificación completo:
              precision    recall  f1-score   support

           1       0.50      0.62      0.55       170
           2       0.33      0.35      0.34        98
           X       0.17      0.08      0.11        86

    accuracy                           0.41       354
   macro avg       0.33      0.35      0.33       354
weighted avg       0.37      0.41      0.39       354



### GOLES DEL EQUIPO LOCAL

In [220]:
# === División del dataset en entrenamiento y prueba (80% / 20%) ===
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y_resultado_local, test_size=0.2, random_state=42)

In [221]:
# === Definición del modelo de árbol de decisión ===
model = DecisionTreeClassifier(random_state=42)

In [222]:
# === Definición del grid de hiperparámetros para optimización ===
param_grid = {
    'max_depth': [3, 5, 7],
    'min_samples_split': [2, 5, 10] 
}

In [223]:
# === Búsqueda de los mejores hiperparámetros con validación cruzada ===
grid_search = GridSearchCV(model, param_grid, cv=5, scoring='accuracy', n_jobs=-1)
grid_search.fit(X_train, y_train)

c:\Users\Miguel\anaconda3\Lib\site-packages\sklearn\model_selection\_split.py:700: UserWarning: The least populated class in y has only 3 members, which is less than n_splits=5.
  warnings.warn(


GridSearchCV(cv=5, estimator=DecisionTreeClassifier(random_state=42), n_jobs=-1,
             param_grid={'max_depth': [3, 5, 7],
                         'min_samples_split': [2, 5, 10]},
             scoring='accuracy')

In [224]:
# === Extracción y visualización de los mejores hiperparámetros ===
best_params = grid_search.best_params_
print("Mejores hiperparámetros:", best_params)

Mejores hiperparámetros: {'max_depth': 5, 'min_samples_split': 5}


In [225]:
# === Entrenamiento del modelo con los mejores hiperparámetros ===
best_model = DecisionTreeClassifier(**best_params, random_state=42)
best_model.fit(X_train, y_train)

DecisionTreeClassifier(max_depth=5, min_samples_split=5, random_state=42)

In [226]:
# === Evaluación del modelo sobre el conjunto de prueba ===
y_test_pred = best_model.predict(X_test)
test_accuracy = accuracy_score(y_test, y_test_pred)

In [227]:
# === Cálculo de la importancia de las variables ===
feature_names = X.columns
importances = best_model.feature_importances_
feature_importance_df = pd.DataFrame({
    'Variable': feature_names,
    'Importancia': importances
}).sort_values(by='Importancia', ascending=False)

In [228]:
# === Mostrar las 10 variables más importantes ===
print(feature_importance_df.head(10))

                                            Variable  Importancia
12                  proporcion local puntos en sitio     0.098343
40      proporcion visitante goles marcados en sitio     0.078071
84             proporcion local total tiros en sitio     0.074946
31           porcentaje visitante mas 4,5 en general     0.072848
87       proporcion visitante total tiros en general     0.072597
7            porcentaje visitante ganados en general     0.065007
85           proporcion local total tiros en general     0.063614
57  porcentaje visitante mas 0,5 marcados en general     0.061910
71         proporcion visitante amarillas en general     0.059304
25               porcentaje local mas 3,5 en general     0.057521


In [229]:
# === Suma de la importancia de variables relacionadas con la rotación ===
cambio_vars = feature_importance_df[feature_importance_df['Variable'].str.contains('cambio', case=False)]
suma_importancia_cambio = cambio_vars['Importancia'].sum()
print(f"Suma total de importancia de las variables relacionadas con la rotación: {suma_importancia_cambio:.4f}")

Suma total de importancia de las variables relacionadas con la rotación: 0.1563


In [230]:
# Matriz de confusión
# Calculamos la matriz de confusión para evaluar los aciertos por clase
conf_matrix = confusion_matrix(y_test, y_test_pred)
print("Matriz de confusión:")
print(conf_matrix)

# F1-score promedio (macro: media sin ponderar entre clases)
f1 = f1_score(y_test, y_test_pred, average='macro')
print(f"F1-score (macro): {f1:.4f}")

# Informe completo con precisión, recall y F1 por clase
print("Informe de clasificación completo:")
# Mostramos un resumen completo de métricas: precisión, recall y F1-score
print(classification_report(y_test, y_test_pred))

Matriz de confusión:
[[ 0 60 12  0  0  0  0  0]
 [16 79 29  0  0  0  0  0]
 [ 6 54 21  0  0  0  0  0]
 [ 3 34 14  0  0  0  0  0]
 [ 2 11  4  0  0  0  0  0]
 [ 0  5  2  0  0  0  0  0]
 [ 0  1  0  0  0  0  0  0]
 [ 0  1  0  0  0  0  0  0]]
F1-score (macro): 0.0857
Informe de clasificación completo:
              precision    recall  f1-score   support

           0       0.00      0.00      0.00        72
           1       0.32      0.64      0.43       124
           2       0.26      0.26      0.26        81
           3       0.00      0.00      0.00        51
           4       0.00      0.00      0.00        17
           5       0.00      0.00      0.00         7
           6       0.00      0.00      0.00         1
           7       0.00      0.00      0.00         1

    accuracy                           0.28       354
   macro avg       0.07      0.11      0.09       354
weighted avg       0.17      0.28      0.21       354



c:\Users\Miguel\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\Miguel\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\Miguel\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


### GOLES EQUIPO VISITANTE

In [231]:
# === División del dataset en entrenamiento y prueba (80% / 20%) ===
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y_resultado_visitante, test_size=0.2, random_state=42)

In [232]:
# === Definición del modelo de árbol de decisión ===
model = DecisionTreeClassifier(random_state=42)

In [233]:
# === Definición del grid de hiperparámetros para optimización ===
param_grid = {
    'max_depth': [3, 5, 7],
    'min_samples_split': [2, 5, 10]
}

In [234]:
# === Búsqueda de los mejores hiperparámetros con validación cruzada ===
grid_search = GridSearchCV(model, param_grid, cv=5, scoring='accuracy', n_jobs=-1)
grid_search.fit(X_train, y_train)

c:\Users\Miguel\anaconda3\Lib\site-packages\sklearn\model_selection\_split.py:700: UserWarning: The least populated class in y has only 4 members, which is less than n_splits=5.
  warnings.warn(


GridSearchCV(cv=5, estimator=DecisionTreeClassifier(random_state=42), n_jobs=-1,
             param_grid={'max_depth': [3, 5, 7],
                         'min_samples_split': [2, 5, 10]},
             scoring='accuracy')

In [235]:
# === Extracción y visualización de los mejores hiperparámetros ===
best_params = grid_search.best_params_
print("Mejores hiperparámetros:", best_params)

Mejores hiperparámetros: {'max_depth': 3, 'min_samples_split': 2}


In [236]:
# === Entrenamiento del modelo con los mejores hiperparámetros ===
best_model = DecisionTreeClassifier(**best_params, random_state=42)
best_model.fit(X_train, y_train)

DecisionTreeClassifier(max_depth=3, random_state=42)

In [237]:
# === Evaluación del modelo sobre el conjunto de prueba ===
y_test_pred = best_model.predict(X_test)
test_accuracy = accuracy_score(y_test, y_test_pred)

In [238]:
# === Cálculo de la importancia de las variables ===
feature_names = X.columns
importances = best_model.feature_importances_
feature_importance_df = pd.DataFrame({
    'Variable': feature_names,
    'Importancia': importances
}).sort_values(by='Importancia', ascending=False)

In [239]:
# === Mostrar las 10 variables más importantes ===
print(feature_importance_df.head(10))

                                              Variable  Importancia
41      proporcion visitante goles marcados en general     0.194419
4                   porcentaje local perdidos en sitio     0.157112
103  proporcion visitante cambios amarillas en general     0.150720
89         proporcion local corners a favor en general     0.132899
101      proporcion local cambios amarillas en general     0.128223
119  proporcion visitante cambios delanteros a cent...     0.118438
84               proporcion local total tiros en sitio     0.118190
115         media visitante cambios minutos en general     0.000000
110   proporcion visitante cambios asistentes en sitio     0.000000
111  proporcion visitante cambios asistentes en gen...     0.000000


In [240]:
# === Suma de la importancia de variables relacionadas con la rotación ===
cambio_vars = feature_importance_df[feature_importance_df['Variable'].str.contains('cambio', case=False)]
suma_importancia_cambio = cambio_vars['Importancia'].sum()
print(f"Suma total de importancia de las variables relacionadas con la rotación: {suma_importancia_cambio:.4f}")

Suma total de importancia de las variables relacionadas con la rotación: 0.3974


In [241]:
# Matriz de confusión
# Calculamos la matriz de confusión para evaluar los aciertos por clase
conf_matrix = confusion_matrix(y_test, y_test_pred)
print("Matriz de confusión:")
print(conf_matrix)

# F1-score promedio (macro: media sin ponderar entre clases)
f1 = f1_score(y_test, y_test_pred, average='macro')
print(f"F1-score (macro): {f1:.4f}")

# Informe completo con precisión, recall y F1 por clase
print("Informe de clasificación completo:")
# Mostramos un resumen completo de métricas: precisión, recall y F1-score
print(classification_report(y_test, y_test_pred))

Matriz de confusión:
[[  5 111   0   0   0   0]
 [  3 117   1   0   0   0]
 [  4  76   0   0   0   0]
 [  0  28   0   0   0   0]
 [  0   6   0   0   0   0]
 [  0   3   0   0   0   0]]
F1-score (macro): 0.0974
Informe de clasificación completo:
              precision    recall  f1-score   support

           0       0.42      0.04      0.08       116
           1       0.34      0.97      0.51       121
           2       0.00      0.00      0.00        80
           3       0.00      0.00      0.00        28
           4       0.00      0.00      0.00         6
           5       0.00      0.00      0.00         3

    accuracy                           0.34       354
   macro avg       0.13      0.17      0.10       354
weighted avg       0.25      0.34      0.20       354



c:\Users\Miguel\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\Miguel\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\Miguel\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
